In [321]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np


**Se cargan los installs, se convierte a fecha la columna created y se la usa como indice**

In [322]:
installs = pd.read_csv('data/installs.csv.gzip', low_memory=False, compression='gzip', parse_dates = ['created'], index_col=['created'])
installs = installs.sort_index()
installs.head()

,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,ip_address,device_language
created,,,,,,,,,,,,,,,,,
2019-04-18 00:00:01.560,70,1891515180541284343,4432995619177048534,NaN,False,False,6287817205707153877,NaN,NaN,Apsalar-Postback,NaN,NaN,NaN,NaN,NaN,8409380583076227678,NaN
2019-04-18 00:00:01.851,70,1891515180541284343,5904733559638204455,NaN,False,False,6287817205707153877,NaN,NaN,Apsalar-Postback,NaN,NaN,NaN,NaN,NaN,806493034070542041,NaN
2019-04-18 00:00:05.152,65,1891515180541284343,896373747754111825,NaN,False,True,6287817205707153877,3.083059e+17,5.274185e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 9; SM-G9650 Bu...,8c8af5e3-96e7-4a49-9f17-cafa7f300f2c,af_app_opened,False,NaN,6319394158544413414,6.977049e+18
2019-04-18 00:00:05.589,27,1494519392962156891,3399210824535017892,NaN,False,False,6287817205707153877,NaN,6.794880e+18,http-kit/2.0,trivago/216 CFNetwork/978.0.7 Darwin/18.5.0,NaN,NaN,True,NaN,4206683315615624585,5.221863e+18
2019-04-18 00:00:06.795,339,1494519392962156891,1541425881979513687,NaN,False,False,6287817205707153877,NaN,6.794880e+18,http-kit/2.0,TikTok/109005 CFNetwork/758.5.3 Darwin/15.6.0,NaN,NaN,True,NaN,8335834010411202437,7.528974e+18


**Se agrupa por usuario y se queda con el primer install de la ventana. Luego se convierte a segundos.**

In [323]:
label1 = installs['2019-04-21':'2019-04-23'].reset_index().groupby('ref_hash').agg({'created' : 'min'}).reset_index()

In [324]:
label1['created'] = label1['created'] - pd.Timestamp('2019-04-21 00:00:00')
label1['created'] = label1['created'].transform(lambda x: x.total_seconds())

#### Ver esto, ¿deberia ser inner el join? Porque asi el modelo solo aprende de usuarios que van convertir

In [325]:
label1 = label1.merge(installs['2019-04-18':'2019-04-20'].reset_index()["ref_hash"],on="ref_hash", how="right")
label1.head(20)

,ref_hash,created
0,2493454867357963,19452.720
1,3589607969461633,108671.325
2,3589607969461633,108671.325
3,3861795021724487,252890.111
4,6455321535851721,217290.613
5,7944594974253876,220770.040
6,14664043499550181,97507.843
7,15057990565785689,238575.817
8,15057990565785689,238575.817
9,15695176139879422,239972.035


**A los que no aparecieron en la ventana de testing les asigno el valor maximo de tiempo: 3 dias(en segundos)**

In [318]:
label1["created"] = label1["created"].fillna(60*60*24*3)
label1 = label1.rename(columns={'created':'label'})

In [319]:
y=label1.loc[:,'label']
y


0          19452.720
1         108671.325
2         108671.325
3         252890.111
4         217290.613
5         220770.040
6          97507.843
7         238575.817
8         238575.817
9         239972.035
10        170757.389
11        243918.623
12         22624.206
13        225764.103
14        225764.103
15        107517.631
16        152442.672
17        152442.672
18         93014.879
19        245073.483
20        166275.213
21         11104.805
22         11104.805
23         11104.805
24          3211.751
25         48230.003
26         48230.003
27         48230.003
28        189169.026
29        103646.069
             ...    
160831    259200.000
160832    259200.000
160833    259200.000
160834    259200.000
160835    259200.000
160836    259200.000
160837    259200.000
160838    259200.000
160839    259200.000
160840    259200.000
160841    259200.000
160842    259200.000
160843    259200.000
160844    259200.000
160845    259200.000
160846    259200.000
160847    259

In [320]:
# data1=label1.drop('label',axis=1)
# data1.head()
#Aca se debe hacer limpieza de los datos... dejar features que sirven y hacer nuevos

,ref_hash
0,2493454867357963
1,3589607969461633
2,3589607969461633
3,3861795021724487
4,6455321535851721


In [268]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4947 entries, 0 to 4946
Data columns (total 18 columns):
ref_hash              4947 non-null int64
created               4947 non-null datetime64[ns]
application_id        4947 non-null int64
ref_type              4947 non-null int64
click_hash            18 non-null object
attributed            4947 non-null bool
implicit              4947 non-null bool
device_countrycode    4947 non-null int64
device_brand          2814 non-null float64
device_model          4711 non-null float64
session_user_agent    4820 non-null object
user_agent            3174 non-null object
event_uuid            1160 non-null object
kind                  1160 non-null object
wifi                  2923 non-null object
trans_id              93 non-null object
ip_address            4947 non-null int64
device_language       4716 non-null float64
dtypes: bool(2), datetime64[ns](1), float64(3), int64(5), object(7)
memory usage: 666.7+ KB


In [269]:
#Vamos a dejar las columnas que nos aporten informacion...

In [270]:
# Podemos "inventar" que los que no tienen info son falsos...
data1['wifi']=data1['wifi'].fillna(False)

In [271]:
data1.drop(['created','ref_type','click_hash','session_user_agent','user_agent','kind','trans_id','event_uuid'],axis =1, inplace = True )


In [272]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4947 entries, 0 to 4946
Data columns (total 10 columns):
ref_hash              4947 non-null int64
application_id        4947 non-null int64
attributed            4947 non-null bool
implicit              4947 non-null bool
device_countrycode    4947 non-null int64
device_brand          2814 non-null float64
device_model          4711 non-null float64
wifi                  4947 non-null bool
ip_address            4947 non-null int64
device_language       4716 non-null float64
dtypes: bool(3), float64(3), int64(4)
memory usage: 323.7 KB


In [273]:
y.head(1)

0    19452.72
Name: label, dtype: float64

In [274]:
data1=data1.fillna(0)

In [275]:
data1.shape,y.shape

((4947, 10), (4947,))

**El problema es supervisado?, para cuando tengamos que predecir no vamos a poder ver absolutamente nada de los datos de esos tres dias, habria que ver como hacer el problema supervisado usando solo los datos de una ventana.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(data1, y, test_size=0.2, random_state=123)

In [277]:
reg=linear_model.LinearRegression()

In [278]:
reg.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [279]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((3957, 10), (990, 10), (3957,), (990,))

In [280]:
pred=reg.predict(X_test)

In [281]:
pred.max()

125824.0

In [242]:
pred.shape

(990,)

In [243]:
y_test

3063    243153.456
918     185233.034
655      21086.786
1502     73309.262
934     183573.777
730      11865.516
31      231064.121
3242     92701.601
3386    107372.837
1084    258992.418
1451     96195.132
4927    185807.476
4289      8037.325
4382    200965.498
239     167946.399
1589     21355.258
1945     59522.470
3091    142783.277
4892      6400.576
761      17125.551
748     116016.453
3838      3244.328
3118    175395.043
439      20047.465
2322     66040.100
4559    102165.108
4115     51349.643
3793    205955.717
2022     54555.023
4798    136842.335
           ...    
3855    156376.743
682      14996.289
4284      9722.838
3927    250419.219
2493    223655.174
4812    158080.085
1914     32244.325
2997     19593.091
1824     62256.247
1179     23825.569
1064    105945.850
3649     87210.832
827     190862.372
4928      5038.878
1370    182485.388
2328     78377.696
1459    258542.142
585     219239.485
2676    133308.203
3341     25720.994
2070      6255.778
1033    1759

Square mean error, es la que se usa para calcular el error en kagle. Da mas bonito asi.

In [244]:
# Square mean square error
mean_squared_error(y_test,pred)**0.5
# np.mean((pred-y_test)**2)

74958.64067328689

#Ahora estas predicciones las agregamos como feature para la segunda ventana...

Empezamos con los nuevos labels


In [245]:
label2 = installs['2019-04-24':'2019-04-26'].reset_index().groupby('ref_hash').agg({'created' : 'min'})

In [246]:
label2['created'] = label2['created'] - pd.Timestamp('2019-04-24 00:00:00')
label2['created'] = label2['created'].transform(lambda x: x.total_seconds())

In [247]:
label2 = label2.reset_index()
label2 = label2.merge(installs['2019-04-21':'2019-04-23'].reset_index(),on="ref_hash", how="inner")
label2 = label2.rename(columns={'created_x':'label','created_y':'created'})
label2

,ref_hash,label,created,application_id,ref_type,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,ip_address,device_language
0,622102439689666,979.059,2019-04-23 23:17:02.399,121,1891515180541284343,NaN,False,True,6287817205707153877,2.523246e+18,5.151882e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 6.0.1; LG-K200...,f8b3f19d-c761-4d6a-9f20-4e1832d38146,Open,True,NaN,1848802948798920304,6.977049e+18
1,622102439689666,979.059,2019-04-23 23:17:02.787,121,1891515180541284343,NaN,False,False,6287817205707153877,2.523246e+18,5.151882e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 6.0.1; LG-K200...,NaN,NaN,True,NaN,1848802948798920304,6.977049e+18
2,928188758790177,247212.597,2019-04-21 01:50:16.400,68,1494519392962156891,NaN,False,False,6287817205707153877,NaN,6.794880e+18,http-kit/2.0,Grability/17147 CFNetwork/978.0.7 Darwin/18.5.0,NaN,NaN,True,NaN,3483843913495440783,8.441417e+18
3,4129436938177104,229521.524,2019-04-21 20:56:18.807,117,1891515180541284343,NaN,False,False,6287817205707153877,NaN,2.399584e+18,adjust.com,NaN,NaN,NaN,NaN,NaN,5840064274212175308,3.301378e+18
4,5133475695351500,209187.715,2019-04-22 23:38:06.589,194,1494519392962156891,NaN,False,False,6287817205707153877,NaN,6.794880e+18,http-kit/2.0,Gardenscapes/3.3.2 CFNetwork/978.0.7 Darwin/18...,NaN,NaN,True,NaN,2437566759560000641,8.441417e+18
5,7881267125156197,176402.652,2019-04-22 14:15:28.970,210,1891515180541284343,NaN,False,False,6287817205707153877,NaN,8.479522e+18,http-kit/2.0,Mozilla/5.0 (Linux; Android 9; SM-G970F Build/...,NaN,NaN,NaN,NaN,97224138037576704,3.301378e+18
6,7944594974253876,9055.796,2019-04-23 13:19:30.040,281,1891515180541284343,NaN,False,False,6287817205707153877,NaN,NaN,Dalvik/2.1.0 (Linux; U; Android 9; SM-N960U1 B...,NaN,NaN,NaN,NaN,NaN,4063933476320529056,NaN
7,13912050945871915,52090.338,2019-04-23 12:54:17.555,226,1891515180541284343,NaN,False,False,6287817205707153877,4.567867e+18,1.292000e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 8.0.0; BLADE V...,NaN,NaN,False,NaN,3645808844944603776,6.977049e+18
8,24294411704078426,182902.601,2019-04-23 03:02:23.309,36,1891515180541284343,NaN,False,False,6287817205707153877,3.083059e+17,9.186120e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 6.0.1; SM-G532...,NaN,NaN,True,NaN,4950519528220185340,6.977049e+18
9,24625906561621678,225275.348,2019-04-22 15:08:28.648,226,1891515180541284343,NaN,False,False,6287817205707153877,2.208835e+18,4.073186e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 8.1.0; moto e5...,NaN,NaN,False,NaN,653240972862608991,6.977049e+18


In [248]:
y2=label2.loc[:,'label']


In [249]:
data2=label2.drop('label',axis=1)
data2.head()

,ref_hash,created,application_id,ref_type,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,ip_address,device_language
0,622102439689666,2019-04-23 23:17:02.399,121,1891515180541284343,NaN,False,True,6287817205707153877,2.523246e+18,5.151882e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 6.0.1; LG-K200...,f8b3f19d-c761-4d6a-9f20-4e1832d38146,Open,True,NaN,1848802948798920304,6.977049e+18
1,622102439689666,2019-04-23 23:17:02.787,121,1891515180541284343,NaN,False,False,6287817205707153877,2.523246e+18,5.151882e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 6.0.1; LG-K200...,NaN,NaN,True,NaN,1848802948798920304,6.977049e+18
2,928188758790177,2019-04-21 01:50:16.400,68,1494519392962156891,NaN,False,False,6287817205707153877,NaN,6.794880e+18,http-kit/2.0,Grability/17147 CFNetwork/978.0.7 Darwin/18.5.0,NaN,NaN,True,NaN,3483843913495440783,8.441417e+18
3,4129436938177104,2019-04-21 20:56:18.807,117,1891515180541284343,NaN,False,False,6287817205707153877,NaN,2.399584e+18,adjust.com,NaN,NaN,NaN,NaN,NaN,5840064274212175308,3.301378e+18
4,5133475695351500,2019-04-22 23:38:06.589,194,1494519392962156891,NaN,False,False,6287817205707153877,NaN,6.794880e+18,http-kit/2.0,Gardenscapes/3.3.2 CFNetwork/978.0.7 Darwin/18...,NaN,NaN,True,NaN,2437566759560000641,8.441417e+18


In [250]:
data2.shape,pred.shape

((4877, 18), (990,))

In [251]:
pred

array([117760., 121984., 119936., 119424., 116864., 116224., 123520.,
       117504., 116480., 121216., 118016., 112128., 116096., 112768.,
       117248., 121472., 121344., 116608., 113408., 115200., 121600.,
       117120., 117504., 119808., 118400., 113920., 117760., 113152.,
       123008., 115968., 117376., 114944., 121344., 121088., 117248.,
       117248., 119680., 112384., 118016., 112768., 117632., 112768.,
       114944., 117248., 118784., 115840., 119296., 118784., 111744.,
       120576., 117120., 115968., 118912., 116480., 113920., 118016.,
       124288., 115968., 116864., 119424., 120832., 121856., 115456.,
       119040., 117760., 116352., 116736., 114816., 113280., 111360.,
       117888., 118016., 116352., 121984., 117376., 118272., 120832.,
       120832., 114944., 117376., 124544., 118528., 113664., 120192.,
       120448., 122112., 121728., 118016., 114432., 114432., 116864.,
       117120., 123904., 123904., 115456., 116736., 118528., 121856.,
       117632., 1140

In [252]:
#Esas pred se las tenemos que agregar al set de entrenamiento ...
#¿Como hacemos? solo tenemos 990 filas! rellenamos con algun numero? o sea 999999?
#Tambien podemos usar el label de la anterior... para esta
data2['label1']=y
data2.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4877 entries, 0 to 4876
Data columns (total 19 columns):
ref_hash              4877 non-null int64
created               4877 non-null datetime64[ns]
application_id        4877 non-null int64
ref_type              4877 non-null int64
click_hash            13 non-null object
attributed            4877 non-null bool
implicit              4877 non-null bool
device_countrycode    4877 non-null int64
device_brand          2709 non-null float64
device_model          4571 non-null float64
session_user_agent    4755 non-null object
user_agent            3262 non-null object
event_uuid            1252 non-null object
kind                  1252 non-null object
wifi                  2919 non-null object
trans_id              89 non-null object
ip_address            4877 non-null int64
device_language       4571 non-null float64
label1                4877 non-null float64
dtypes: bool(2), datetime64[ns](1), float64(4), int64(5), object(7)
memory us

In [253]:
data2.drop(['created','ref_type','click_hash','session_user_agent','user_agent','kind','trans_id','event_uuid'],axis =1, inplace = True )
data2.fillna(0)


,ref_hash,application_id,attributed,implicit,device_countrycode,device_brand,device_model,wifi,ip_address,device_language,label1
0,622102439689666,121,False,True,6287817205707153877,2.523246e+18,5.151882e+18,True,1848802948798920304,6.977049e+18,19452.720
1,622102439689666,121,False,False,6287817205707153877,2.523246e+18,5.151882e+18,True,1848802948798920304,6.977049e+18,108671.325
2,928188758790177,68,False,False,6287817205707153877,0.000000e+00,6.794880e+18,True,3483843913495440783,8.441417e+18,108671.325
3,4129436938177104,117,False,False,6287817205707153877,0.000000e+00,2.399584e+18,0,5840064274212175308,3.301378e+18,252890.111
4,5133475695351500,194,False,False,6287817205707153877,0.000000e+00,6.794880e+18,True,2437566759560000641,8.441417e+18,217290.613
5,7881267125156197,210,False,False,6287817205707153877,0.000000e+00,8.479522e+18,0,97224138037576704,3.301378e+18,220770.040
6,7944594974253876,281,False,False,6287817205707153877,0.000000e+00,0.000000e+00,0,4063933476320529056,0.000000e+00,97507.843
7,13912050945871915,226,False,False,6287817205707153877,4.567867e+18,1.292000e+18,False,3645808844944603776,6.977049e+18,238575.817
8,24294411704078426,36,False,False,6287817205707153877,3.083059e+17,9.186120e+18,True,4950519528220185340,6.977049e+18,238575.817
9,24625906561621678,226,False,False,6287817205707153877,2.208835e+18,4.073186e+18,False,653240972862608991,6.977049e+18,239972.035


In [254]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4877 entries, 0 to 4876
Data columns (total 11 columns):
ref_hash              4877 non-null int64
application_id        4877 non-null int64
attributed            4877 non-null bool
implicit              4877 non-null bool
device_countrycode    4877 non-null int64
device_brand          2709 non-null float64
device_model          4571 non-null float64
wifi                  2919 non-null object
ip_address            4877 non-null int64
device_language       4571 non-null float64
label1                4877 non-null float64
dtypes: bool(2), float64(4), int64(4), object(1)
memory usage: 390.5+ KB


In [255]:
data2.shape,y2.shape

((4877, 11), (4877,))

In [256]:
X2_train, X2_test, y2_train, y2_test = \
    train_test_split(data2, y2, test_size=0.2, random_state=123)

In [257]:
reg2=linear_model.LinearRegression()


In [258]:
X2_train=X2_train.fillna(0)
y2_train=y2_train.fillna(0)
X2_test=X2_test.fillna(0)


In [259]:
reg2.fit(X2_train,y2_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [260]:
X2_train.shape,X2_test.shape,y2_train.shape,y2_test.shape
pred2=reg2.predict(X2_test)
mean_squared_error(y2_test,pred2)**0.5
# np.mean((pred2-y2_test)**2)


75613.49934702167